In [1]:
%matplotlib inline

In [2]:
from scipy import stats as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import warnings
warnings.filterwarnings('ignore')

sns.set_context("poster")

In [3]:
train = pd.read_csv('data/test.csv')

##add explicit event_id to merge on - ordering should be the same but better not take the risk
train['event_id'] = [x for x in xrange(len(train))]

#train[['event_id','hotel_cluster']].to_csv('data/train_sample_y.csv',index=False)
#train.drop('hotel_cluster',1,inplace=True)
train.to_csv('data/test_x.csv',index=False)

In [4]:
train_x = pd.read_csv('data/test_x.csv')
train_x.columns

Index([u'id', u'date_time', u'site_name', u'posa_continent',
       u'user_location_country', u'user_location_region',
       u'user_location_city', u'orig_destination_distance', u'user_id',
       u'is_mobile', u'is_package', u'channel', u'srch_ci', u'srch_co',
       u'srch_adults_cnt', u'srch_children_cnt', u'srch_rm_cnt',
       u'srch_destination_id', u'srch_destination_type_id', u'hotel_continent',
       u'hotel_country', u'hotel_market', u'event_id'],
      dtype='object')

What do we want to check?
- data type
- no. uniques
- null values

In [5]:
def desc_df(dataf):
    lentrain = len(dataf)
    outp = []
    for i, j in train_x.dtypes.iteritems():
        outp.append([i, j, float(len(dataf[i].dropna()))/lentrain, len(dataf[i].unique())])

    return pd.DataFrame(outp,columns = ['variable','type','% non null','unique values'])

In [6]:
outp = desc_df(train_x)
outp

,variable,type,% non null,unique values
0,id,int64,1.000000,2528243
1,date_time,object,1.000000,2399366
2,site_name,int64,1.000000,53
3,posa_continent,int64,1.000000,5
4,user_location_country,int64,1.000000,236
5,user_location_region,int64,1.000000,988
6,user_location_city,int64,1.000000,36722
7,orig_destination_distance,float64,0.664802,1344827
8,user_id,int64,1.000000,1181577
9,is_mobile,int64,1.000000,2


In [7]:
train_x[['date_time','srch_ci','srch_co','orig_destination_distance']].head()

,date_time,srch_ci,srch_co,orig_destination_distance
0,2015-09-03 17:09:54,2016-05-19,2016-05-23,5539.0567
1,2015-09-24 17:38:35,2016-05-12,2016-05-15,5873.2923
2,2015-06-07 15:53:02,2015-07-26,2015-07-27,3975.9776
3,2015-09-14 14:49:10,2015-09-14,2015-09-16,1508.5975
4,2015-07-17 09:32:04,2015-07-22,2015-07-23,66.7913


Looks like for the data manipulation we need to:

* binarize the 'int64' variables
* Create new date variables for the date_times (this will also deal with any null values)
* Keep orig_destination_distance and cnt as is - need to deal with null values
    * Potentially we create bands and all 0's in band deals with null values

In [8]:
def manip(x):
    if x['unique values'] == 2:
        return 'bool'
    if x['variable'] in ['user_id','srch_destination_id','event_id']:
        return 'id'
    if x['type'] == 'int64' and x['variable'] != 'cnt':
        return 'cat'
    if x['variable'] in ['cnt','orig_destination_distance']:
        return 'con'
    if x['variable'] == 'date_time':
        return 'datetime'
    if x['variable'] in ['srch_ci','srch_co']:
        return 'date'

outp['manip_type'] = outp.apply(lambda x: manip(x),1)

In [9]:
outp

,variable,type,% non null,unique values,manip_type
0,id,int64,1.000000,2528243,cat
1,date_time,object,1.000000,2399366,datetime
2,site_name,int64,1.000000,53,cat
3,posa_continent,int64,1.000000,5,cat
4,user_location_country,int64,1.000000,236,cat
5,user_location_region,int64,1.000000,988,cat
6,user_location_city,int64,1.000000,36722,cat
7,orig_destination_distance,float64,0.664802,1344827,con
8,user_id,int64,1.000000,1181577,id
9,is_mobile,int64,1.000000,2,bool


### Data manipulation - Creating the final dataset

I need a function that takes each categorization and does something to the data
* cat - binarise the data
* con - as is
* bool - as is
* date - take out date parts and binarize
* datetime - as above with more date parts

Lets build some binarize function to help us - keeping it in pandas rather than using base sklearn

Now lets look at the dates to create categorical variables and then binarise everything together

For dates we want:

* day
* week of year
* day of week
* month
* year

For datetimes we additinally want:

* hour
* am/pm

In [10]:
from datetime import datetime, date
import math

def date_parts_df(data,variable,iid='event_id',dformat='%Y-%m-%d'):
    new_df = data[[iid,variable]]

    new_df[variable+'_year'] = new_df[variable].apply(
        lambda x: datetime.strptime(x,dformat).year if pd.notnull(x) else None
    )

    new_df[variable+'_day'] = new_df[variable].apply(
        lambda x: datetime.strptime(x,dformat).day if pd.notnull(x) else None
    )

    new_df[variable+'_month'] = new_df[variable].apply(
        lambda x: datetime.strptime(x,dformat).month if pd.notnull(x) else None
    )

    new_df[variable+'_dayofweek'] = new_df[variable].apply(
        lambda x: datetime.strptime(x,dformat).weekday() if pd.notnull(x) else None
    )

    new_df[variable+'_yearquarter'] = new_df[variable].apply(
        lambda x: (datetime.strptime(x,dformat).toordinal() - date(datetime.strptime(x,dformat).year, 1, 1).toordinal() + 1)*4/365 if pd.notnull(x) else None
    )

    return new_df.drop(variable,1)

def datetime_parts_df(data,variable,iid='user_id',dformat='%Y-%m-%d %H:%M:%S'):
    new_df = date_parts_df(data,variable,iid,dformat=dformat)
    
    new_df[variable+'_hour'] = data[variable].apply(
        lambda x: datetime.strptime(x,dformat).hour if pd.notnull(x) else None
    )
    
    return new_df

Order of manipulation:

* Change date and datetime fields
* merge to categorical fields
* create all as categroical fields
* merge to boolean and continuous

In [21]:
## Extra issue of dates being rubbish!
def null_if_not_date(var,dformat):
    nvar = []
    for i in var:
        try:
            nvar.append(datetime.strptime(i,dformat))
        except:
            nvar.append(None)
    return nvar

train_x['srch_ci'] = train_x['srch_ci'].apply(null_if_not_date,dformat='%Y-%m-%d')
train_x['srch_co'] = train_x['srch_ci'].apply(null_if_not_date,dformat='%Y-%m-%d')

TypeError: 'float' object is not iterable

In [ ]:
print train_x['srch_ci'].min(), train_x['srch_ci'].max()
print train_x['srch_co'].min(), train_x['srch_co'].max()

In [11]:
##Get all date parts for dates and datetimes
i=0
for var in outp[outp['manip_type']=='date']['variable']:
    if i==0:
        dates_df = date_parts_df(train_x,var,iid='event_id')
        i+=1
    else:
        df2 = date_parts_df(train_x,var,iid='event_id')
#        print dates_df.head()
#        print df2.head()
        dates_df = pd.merge(dates_df,df2,on='event_id')

for var in outp[outp['manip_type']=='datetime']['variable']:
    df2 = datetime_parts_df(train_x,var,iid='event_id')
    dates_df = pd.merge(dates_df,df2,on='event_id')

print dates_df.head()

ValueError: time data '2161-10-00' does not match format '%Y-%m-%d'

In [ ]:
###Append all categorical variables
cat_df = train_x[['event_id']+outp[outp['manip_type']=='cat']['variable'].tolist()]

full_cat_df = pd.merge(dates_df,cat_df,on='event_id')

In [ ]:
full_cat_df.head()

In [ ]:
##Add back ids from original data
full_cat_df = pd.merge(
    train_x[outp[outp['manip_type']=='id']['variable'].tolist()],
    full_cat_df,
    on='event_id'
    )

In [ ]:
full_cat_df.head()

In [ ]:
### Save final file
full_cat_df.to_csv('data/test_complete_df.csv')